# Pryngles module

## Module base

Goals of the module:
- Future `base`
- Basic Classes and functions of the Class.
- It is based on existing infrastructure.

In [6]:
##HEADER
from pryngles import *
#import rebound as rb

In [174]:
class Object(PrynglesCommon):
    """
    Global class of object
    """    
    pass

In [175]:
class Star(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 m=1,R=1,
                 Prot=1,
                 N=1000
                ):
        #Hash of the object
        self.hash=hash

        #Center of the system (object)
        self.center=center

        #Basic common properties
        self.m=m #mass [cu]
        self.R=R #radius [cu]

        #Dynamic parameters
        self.Prot=Prot

        #Sampling parameters
        self.N=N #number of spangles
        
        #Specific attributes

    def updateObject(self,**props):
        self.__dict__.update(props)

class Planet(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 m=1,R=1,
                 Prot=1,
                 N=1000
                ):
        #Hash of the object
        self.hash=hash

        #Center of the system (object)
        self.center=center

        #Basic common properties
        self.m=m #mass [cu]
        self.R=R #radius [cu]

        #Dynamic parameters
        self.Prot=Prot

        #Sampling parameters
        self.N=N #number of spangles
        
        #Specific attributes
        
    def updateObject(self,**props):
        self.__dict__.update(props)

        
class Ring(Object):
    """
    Creates a star.
    """
    def __init__(self,
                 hash=None,
                 center=None,
                 fi=1.5,fe=2.5,
                 N=1000
                ):
        #Hash of the object
        self.hash=hash

        #Center of the system (object)
        if center is None:
            raise ValueError(f"You must provide a valid object for the center.  {center} provided")
        self.center=center

        #Basic common properties
        self.fi=fi
        self.fe=fe

        #Sampling parameters
        self.N=N #number of spangles
        
        #Update properties
        self.updateObject(**self.__dict__)
        
    def updateObject(self,**props):
        self.__dict__.update(props)
        
        #Derivative properties
        self.ri=self.fi*self.center.R
        self.re=self.fe*self.center.R

In [176]:
S=Planet(m=2,R=0.1)
P=Planet(R=0.1)
R=Ring(center=P,fi=1)
R.updateObject(fi=2)
print(S)
print(P)
print(R)

{'hash': None, 'center': None, 'm': 2, 'R': 0.1, 'Prot': 1, 'N': 1000}
{'hash': None, 'center': None, 'm': 1, 'R': 0.1, 'Prot': 1, 'N': 1000}
{'hash': None, 'center': <__main__.Planet object at 0x7fb758518dc0>, 'fi': 2, 'fe': 2.5, 'N': 1000, 'ri': 0.2, 're': 0.25}


In [148]:
S=Star()


In [98]:
S.show(),P.show()

{'hash': None, 'center': '0', 'm': 1, 'R': 1, 'Prot': 1, 'N': 1000}
{'hash': None, 'center': '0', 'm': 1, 'R': 1, 'Prot': 1, 'N': 1000}


(None, None)

In [9]:
System_doc=\
"""
Creates a planetary system.

Examples:

    sys=System()
    sys=System(units=['msun','km','s'])
    sys=System(stars=dict(M=1,R=1))
    
Initialization (primary) attributes:

    units = ['msun','au','yr']: 
        Units used in calculations following the conventions and signs of rebound. List [3].
        
    stars = None: 
        Stellar in the system.    
    planets = None: 
        Planets in the system.  Planet include 'moons'.
    rings = None: 
        Rings in the system.
           
        NOTE: Objects are provided either by description or as an object. Those attributes can be lists or 
              a single value.
    
    rebound = True:
        Set True if you want to simulte the orbit of objects (for instance if you want to calculate
        TTVs).  If False it will calculate orbits using Keplerian dynamics.

Other Public (secondary) attributes:

    N: Number of objects in the system.

Important private attributes:

    _sim: Rebound simulation object.
""";

In [171]:
class System(object):
    def __init__(self,
                 units=['Msun','au','yr'],
                 stars=None,planets=None,rings=None,
                 rebound=True
                ):
        #Initialize rebound
        self.units=units
        #Stars
        if stars is None:
            pass
        else:
            stars=[Star()]

Example.__doc__=Example_doc

In [172]:
System()

In [65]:
Example_method_doc=\
"""
This is a test method of class Example.

Parameters:
    
"""
def method(self):
    print(self.a)
Example.method=method
Example.method.__doc__=Example_method_doc

--End--

In [66]:
E=Example()

In [67]:
Example?

# Convert

In [60]:
%%javascript
IPython.notebook.kernel.execute('FILE=\"' + IPython.notebook.notebook_name + '\"')

<IPython.core.display.Javascript object>

In [61]:
!make -C .. DEVFILES=dev/{FILE} convert

Converting iPython Notebooks dev/pryngles-_template.ipynb...
Analysing file pryngles-_template.ipynb:
	Directory: src//pryngles
	Filename: _template
	Target object: src//pryngles/_template.py
	Converting from ipynb dev/pryngles-_template.ipynb to python src//pryngles/_template.py...
	Processing magic commands...
	Triming end...
	Triming test...
	Using as template src/.temp
Completed.


In [3]:
# Converted